In [2]:
# imports
from array import array
import struct
import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
def decode(p1,p2,p3,p4):
    try:
        label = open(p2, 'rb')
        imgs = open(p1, 'rb')
        test_imgs = open(p3, 'rb')
        test_labels = open(p4, 'rb')
    except Exception:
        print ("FileNotFound")
        exit()
    
    label.read(8)
    labels = list(array("B", label.read()))
    
    test_labels.read(8)
    t_labels = list(array("B", test_labels.read()))
    
    magic, size, rows, cols = struct.unpack(">IIII", imgs.read(16))
    images = [float(i)/float(255) for i in list(array("B", imgs.read()))]
    
    t_magic, t_size, t_rows, t_cols = struct.unpack(">IIII", test_imgs.read(16))
    t_images = [float(i) / float(255) for i in list(array("B", test_imgs.read()))]
    
    train_data=[(images[i:i+784]) for i in range(0,len(images),784)]
    test_data=[(t_images[i:i+784]) for i in range(0,len(t_images),784)]
    
    return train_data, labels, test_data, t_labels

In [4]:
train_data, train_labels, test_data, test_labels = decode('train-images-idx3-ubyte', 'train-labels-idx1-ubyte', 't10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte')

In [5]:
# Score the model
def show_scoring(model, model_name):
    print("Scoring of %s:\n" % (model_name))
    print("Score: %s\n" % model.score(test_data, test_labels))

    print("Classification report:")
    prediction = model.predict(test_data).astype(np.int32)
    print(classification_report(prediction, test_labels))

    print("\n\n")
    print("Confusion Matrix:\n")
    print("\t", end="")
    for i in range(10):
        print(i, end="\t")
    print('\n\t', end="")
    for i in range(10):
        print('----', end="\t")
    print()
    for i, row in enumerate(confusion_matrix(prediction, test_labels, labels=range(10))):
        print("    %d\t|" % i, end="")
        for cell in row:
            print(cell, end="\t")
        print()

In [7]:
# Run Support Vector Machine
from sklearn import svm
# http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
clf = svm.SVC(kernel='rbf', gamma='auto', tol=0.001, C=1.0, class_weight=None,
  max_iter=-1, random_state=None, shrinking=True)
clf.fit(train_data, train_labels)
show_scoring(clf, "Support Vector Machine")

Scoring of Support Vector Machine:

Score: 0.9446

Classification report:
             precision    recall  f1-score   support

          0       0.99      0.96      0.97      1008
          1       0.99      0.97      0.98      1158
          2       0.93      0.94      0.93      1025
          3       0.94      0.93      0.94      1024
          4       0.96      0.93      0.94      1008
          5       0.91      0.93      0.92       871
          6       0.97      0.95      0.96       978
          7       0.93      0.96      0.94      1000
          8       0.92      0.94      0.93       948
          9       0.92      0.94      0.93       980

avg / total       0.95      0.94      0.94     10000




Confusion Matrix:

	0	1	2	3	4	5	6	7	8	9	
	----	----	----	----	----	----	----	----	----	----	
    0	|967	0	9	1	1	7	9	2	4	8	
    1	|0	1121	1	1	1	4	3	13	7	7	
    2	|2	2	961	15	7	5	4	22	7	0	
    3	|0	2	9	951	0	30	1	5	14	12	
    4	|0	0	10	1	938	7	5	8	8	31	
    5	|5	1	1	15	0	810	9	1	23	6	


In [6]:
# Run Linear Regression
from sklearn.linear_model import LogisticRegression
# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
clf = LogisticRegression(solver='newton-cg', penalty='l2', tol=0.0001, C=1.0, class_weight=None, random_state=None, 
                         max_iter=100, multi_class='ovr', n_jobs=-1)
clf.fit(train_data, train_labels)
show_scoring(clf, "Linear Regression")

Scoring of Linear Regression:

Score: 0.9198

Classification report:
             precision    recall  f1-score   support

          0       0.98      0.95      0.96      1012
          1       0.98      0.96      0.97      1156
          2       0.89      0.94      0.91       982
          3       0.91      0.89      0.90      1027
          4       0.93      0.92      0.93       996
          5       0.86      0.89      0.88       865
          6       0.95      0.94      0.94       968
          7       0.93      0.93      0.93      1024
          8       0.87      0.87      0.87       975
          9       0.89      0.90      0.90       995

avg / total       0.92      0.92      0.92     10000




Confusion Matrix:

	0	1	2	3	4	5	6	7	8	9	
	----	----	----	----	----	----	----	----	----	----	
    0	|959	0	9	4	1	10	9	2	10	8	
    1	|0	1112	8	0	2	2	3	7	14	8	
    2	|1	3	920	17	5	0	7	22	5	2	
    3	|2	1	20	918	3	42	2	5	21	13	
    4	|0	0	9	2	915	10	6	8	15	31	
    5	|5	1	4	23	0	769	20	1	28	14	

In [86]:
from sklearn.neural_network import MLPClassifier
#http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier
clf = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(60), tol=0.0001, learning_rate='constant',
                    activation='relu', alpha=1e-05, beta_1=0.9, beta_2=0.999, early_stopping=False, 
                    epsilon=1e-08, learning_rate_init=0.001, max_iter=200, random_state=None)
clf.fit(train_data, train_labels)
show_scoring(clf, "Multi-layer Perceptron Neural Network")

Scoring of Multi-layer Perceptron Neural Network:

Score: 0.9712

Classification report:
             precision    recall  f1-score   support

          0       0.99      0.97      0.98       994
          1       0.99      0.98      0.99      1139
          2       0.97      0.97      0.97      1038
          3       0.97      0.97      0.97      1015
          4       0.97      0.97      0.97       979
          5       0.96      0.96      0.96       890
          6       0.97      0.97      0.97       952
          7       0.97      0.98      0.97      1020
          8       0.96      0.97      0.96       965
          9       0.96      0.96      0.96      1008

avg / total       0.97      0.97      0.97     10000




Confusion Matrix:

	0	1	2	3	4	5	6	7	8	9	
	----	----	----	----	----	----	----	----	----	----	
    0	|969	0	4	0	1	3	8	1	7	1	
    1	|0	1121	2	0	1	0	3	7	1	4	
    2	|0	5	1005	6	3	0	3	8	6	2	
    3	|1	1	3	982	0	14	1	4	2	7	
    4	|1	0	3	0	951	1	7	1	5	10	
    5	|2	1	1	7	2	858	8